In [1]:
import scipy
matlab = scipy.io.loadmat('Xtrain.mat')

In [2]:
X_train = matlab['Xtrain']

In [3]:
import plotly.express as px
import plotly.graph_objects as go

In [4]:
# Convert the data to a list
X_train_list = X_train.flatten().tolist()
print(X_train_list)

# Plot a line of the values in the training set
fig = px.line(y=X_train_list, x=range(len(X_train_list)), title='Line Plot of X_train Values')
fig.update_layout(xaxis_title='Index', yaxis_title='Value') 
fig.show()

[86, 141, 95, 41, 22, 21, 32, 72, 138, 111, 48, 23, 19, 27, 59, 129, 129, 58, 27, 19, 24, 46, 112, 144, 73, 30, 20, 19, 37, 92, 152, 93, 36, 20, 18, 29, 71, 146, 117, 46, 23, 18, 22, 52, 128, 142, 62, 26, 17, 19, 37, 100, 158, 86, 32, 17, 17, 27, 72, 154, 118, 43, 20, 15, 21, 47, 128, 150, 63, 24, 16, 17, 33, 92, 166, 95, 33, 16, 14, 22, 58, 149, 137, 50, 20, 15, 16, 35, 106, 169, 82, 28, 15, 14, 23, 65, 160, 133, 45, 18, 14, 16, 36, 111, 176, 80, 26, 14, 13, 20, 60, 163, 140, 45, 18, 12, 14, 30, 98, 185, 93, 28, 14, 12, 17, 46, 145, 167, 58, 19, 11, 12, 21, 68, 183, 132, 38, 15, 11, 13, 26, 93, 202, 100, 27, 12, 10, 13, 30, 114, 206, 82, 21, 11, 9, 10, 30, 120, 215, 77, 20, 10, 8, 9, 21, 95, 234, 99, 22, 9, 8, 7, 9, 35, 176, 215, 46, 11, 7, 5, 4, 3, 4, 19, 105, 125, 55, 30, 28, 41, 71, 98, 79, 49, 34, 33, 47, 75, 95, 76, 48, 34, 35, 49, 78, 96, 75, 47, 34, 34, 49, 81, 99, 76, 46, 32, 33, 49, 83, 101, 74, 44, 32, 33, 50, 85, 103, 73, 43, 30, 31, 49, 85, 106, 74, 43, 31, 31, 49, 87, 109

In [5]:
import pandas as pd

In [6]:
def create_features(timeseries_array, n_lags):
    df = pd.DataFrame(timeseries_array, columns=['label'])
    for i in range(1, n_lags + 1):
        df[f'lag {i}'] = df['label'].shift(i)
    df = df.dropna()

    X = df.drop(columns=['label']).to_numpy()
    y = df['label'].to_numpy()

    return X, y

X, y = create_features(X_train_list, 10)

print(X.shape)
print(y.shape)

(990, 10)
(990,)


In [7]:
def expanding_window_cv_sets(X, y, folds: int, validation_split_ratio: float = 0.2):
    
    n = len(X)
    fold_size = int(n / folds)

    for fold in range(folds):
        start_fold = fold * fold_size
        end_fold = start_fold + fold_size

        training_start = 0
        training_end = int(end_fold - fold_size * validation_split_ratio)

        validation_start = training_end

        training_set = X[training_start:training_end], y[training_start:training_end]
        validation_set = X[validation_start:end_fold], y[validation_start:end_fold]
        yield training_set, validation_set

# Example usage
folds = 5
for train_set, val_set in expanding_window_cv_sets(X, y, folds):
    train_X, train_y = train_set
    val_X, val_y = val_set

    print("Training Set:")
    print(train_X.shape, train_y.shape)
    print("Validation Set:")
    print(val_X.shape, val_y.shape)
    print("\n")


Training Set:
(158, 10) (158,)
Validation Set:
(40, 10) (40,)


Training Set:
(356, 10) (356,)
Validation Set:
(40, 10) (40,)


Training Set:
(554, 10) (554,)
Validation Set:
(40, 10) (40,)


Training Set:
(752, 10) (752,)
Validation Set:
(40, 10) (40,)


Training Set:
(950, 10) (950,)
Validation Set:
(40, 10) (40,)




In [8]:
# Create two tables to store the results of CV grid search
epoch_grid_search_results = pd.DataFrame(columns=['epoch', 'fold', 'MSE', 'MAE'])
grid_search_results = pd.DataFrame(columns=['hidden_units', 'lags', 'fold', 'MSE', 'MAE'])

In [9]:
FOLDS = 5
EPOCHS = [10, 20, 50, 100, 500, 1000]
LAGS = [5, 15, 25, 35, 50]
HIDDEN_UNITS = [5, 10, 20, 50, 100]

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

X, y = create_features(X_train_list, 25)

for epoch in EPOCHS:
    print("Epoch: ", epoch)
    for fold, (train_set, val_set) in enumerate(expanding_window_cv_sets(X, y, folds)):

        train_X, train_y = train_set
        val_X, val_y = val_set
        
        # Reshape the data to be 3D for LSTM input
        train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))

        # First, we are gonna fix the LSTM to 10 hidden units and grid search the epochs

        # Define the lSTM model with 15 hidden units
        model = Sequential([
            LSTM(10, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=False),
            Dense(1)
        ])

        model.compile(optimizer='adam', loss='mse')
        model.fit(train_X, train_y, epochs=epoch, batch_size=32, verbose=0)

        # Recursively predict the validation set, so start with the first example of the validation set and use the model to predict the next value, then use that value to predict the next one, and so on.
        current_input = val_X[0].reshape((1, 1, val_X.shape[2]))
        y_pred = []

        for i in range(len(val_y)):
            new_y = model.predict(current_input)
            y_pred.append(new_y[0][0])
            # Remove the first value of the current input and append the new value to the end of the input sequence
            current_input = np.append(current_input[:, :, 1:], new_y.reshape((1, 1, 1)), axis=2)
        
        y_pred = np.array(y_pred).reshape(-1, 1)
        print("Epochs: ", epoch)
        print("Standardized MSE: ", mean_squared_error(val_y, y_pred))
        print("Standardized MAE: ", np.mean(np.abs(val_y - y_pred)))
        print("\n")
        
        # Append the results to the epoch grid search results table
        epoch_grid_search_results = pd.concat([epoch_grid_search_results, pd.DataFrame({'fold': [fold],'epoch': [epoch], 'MSE': [mean_squared_error(val_y, y_pred)], 'MAE': [np.mean(np.abs(val_y - y_pred))]})], ignore_index=True)

2025-05-09 16:44:53.121459: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-09 16:44:53.187367: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-09 16:44:53.234101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746801893.282814 1557765 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746801893.301161 1557765 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746801893.336802 1557765 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Epoch:  10


2025-05-09 16:44:55.787471: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/daan/Documents/GitHub/DL25-Groep-DDDN/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━

/tmp/ipykernel_1557765/3403956020.py:48: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/home/daan/Documents/GitHub/DL25-Groep-DDDN/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━

/home/daan/Documents/GitHub/DL25-Groep-DDDN/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━

/home/daan/Documents/GitHub/DL25-Groep-DDDN/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━

In [11]:
display(epoch_grid_search_results)
epoch_grid_search_results.to_csv('epoch_grid_search_results.csv', index=False)


,epoch,fold,MSE,MAE
0,10,0,3309.845459,46.571754
1,10,1,3537.963867,48.711910
2,10,2,9516.911133,64.563415
3,10,3,898.398865,25.893292
4,10,4,4175.203125,50.139745
5,20,0,3823.041992,48.845609
6,20,1,4407.248535,57.567957
7,20,2,16684.169922,90.883396
8,20,3,1313.935181,29.589207
9,20,4,10383.598633,68.713541


In [12]:
import pandas as pd
epoch_grid_search_results = pd.read_csv('epoch_grid_search_results.csv')

In [13]:
epoch_average = epoch_grid_search_results.groupby('epoch').agg({'MSE': 'mean', 'MAE': 'mean'})
# Write the average results to a CSV file
epoch_average.to_csv('epoch_average.csv', index=False)
epoch_average = pd.read_csv('epoch_average.csv')


In [14]:
BEST_EPOCH = int(epoch_grid_search_results.loc[epoch_grid_search_results['MSE'].idxmin()]['epoch'])
print("Best epoch: ", BEST_EPOCH)

Best epoch:  10


In [15]:

for lags in LAGS:
    print(f"Lag: {lags}")
    X, y = create_features(X_train_list, lags)

    # Expanding window cross-validation
    for fold, (train_set, validation_set) in enumerate(expanding_window_cv_sets(X, y, FOLDS)):
        train_X, train_y = train_set
        val_X, val_y = validation_set

        # Reshape the data to be 3D for LSTM input
        train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
        val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))

        for hidden_units in HIDDEN_UNITS:
            # Define the LSTM model with the current number of hidden units
            model = Sequential([
                LSTM(hidden_units, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=False),
                Dense(1)
            ])

            model.compile(optimizer='adam', loss='mse')
            model.fit(train_X, train_y, epochs=BEST_EPOCH, batch_size=32, verbose=0)

            # Recursively predict the validation set
            current_input = val_X[0].reshape((1, 1, val_X.shape[2]))
            y_pred = []

            for i in range(len(val_y)):
                new_y = model.predict(current_input)
                y_pred.append(new_y[0][0])
                current_input = np.append(current_input[:, :, 1:], new_y.reshape((1, 1, 1)), axis=2)

            y_pred = np.array(y_pred).reshape(-1, 1)
            print(f"Lag: {lags}, Hidden Units: {hidden_units}")
            print("Standardized MSE: ", mean_squared_error(val_y, y_pred))
            print("Standardized MAE: ", np.mean(np.abs(val_y - y_pred)))
            print("\n")

            # Append the results to the lag grid search results table
            grid_search_results = pd.concat([grid_search_results, pd.DataFrame({'fold': [folds], 'lag': [lags], 'hidden_units': [hidden_units], 'MSE': [mean_squared_error(val_y, y_pred)], 'MAE': [np.mean(np.abs(val_y - y_pred))]})], ignore_index=True)

Lag: 5


/home/daan/Documents/GitHub/DL25-Groep-DDDN/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━

/tmp/ipykernel_1557765/1800937931.py:40: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/home/daan/Documents/GitHub/DL25-Groep-DDDN/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━

/home/daan/Documents/GitHub/DL25-Groep-DDDN/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━

/home/daan/Documents/GitHub/DL25-Groep-DDDN/venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━

In [16]:
display(grid_search_results)
grid_search_results.to_csv('grid_search_results.csv', index=False)

average_lag_units = grid_search_results.groupby(['lag', 'hidden_units']).agg({'MSE': 'mean', 'MAE': 'mean'})
display(average_lag_units)
# Write the average results to a CSV file
average_lag_units.to_csv('average_lag_units.csv', index=False)
average = pd.read_csv('average_lag_units.csv')


,hidden_units,lags,fold,MSE,MAE,lag
0,5,NaN,5,6211.751953,56.720813,5.0
1,10,NaN,5,6118.529297,56.113528,5.0
2,20,NaN,5,6018.700195,55.161251,5.0
3,50,NaN,5,5913.758301,54.997351,5.0
4,100,NaN,5,4573.289062,44.016505,5.0
...,...,...,...,...,...,...
120,5,NaN,5,7727.246582,52.286899,50.0
121,10,NaN,5,9522.869141,57.999866,50.0
122,20,NaN,5,8828.887695,55.203668,50.0
123,50,NaN,5,8448.288086,54.621607,50.0


MSE         MAE
lag  hidden_units                          
5.0  5             5.216505e+03   51.617778
     10            4.690366e+03   49.380327
     20            4.979761e+03   53.160162
     50            1.554700e+04   81.464297
     100           1.044456e+05  139.389354
15.0 5             4.686325e+03   53.093196
     10            2.371759e+06  477.483658
     20            4.746730e+03   52.842942
     50            4.764407e+03   57.496930
     100           3.581540e+03   49.324373
25.0 5             4.031505e+03   50.033192
     10            4.458748e+03   49.037948
     20            5.126040e+03   49.707995
     50            6.044405e+03   56.843206
     100           4.737581e+03   49.489232
35.0 5             4.963180e+03   51.813257
     10            4.913510e+03   48.325807
     20            4.654492e+03   46.133495
     50            4.955097e+03   45.421469
     100           5.091843e+03   46.264321
50.0 5             6.657154e+03   63.243818
     10            5.439127e+03   50.241021
     20            5.220586e+03   47.163346
     50            4.836121e+03   46.706234
     100           5.222378e+03   47.172323

In [17]:
grid_search_results = pd.read_csv('grid_search_results.csv')
average_lag_units = pd.read_csv('average_lag_units.csv')

In [18]:
matlab2 = scipy.io.loadmat('Xtest.mat')
X_matlab_test = matlab2['Xtest']
X_test_list = X_matlab_test.flatten().tolist()

concat_train_test = X_train_list + X_test_list
fig = px.line(y=concat_train_test, x=range(len(concat_train_test)), title='Line Plot of X_train and X_test Values')
fig.update_layout(xaxis_title='Index', yaxis_title='Value')
fig.show()

In [39]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.metrics import mean_squared_error

number_of_lags = 35

X, y = create_features(X_train_list, number_of_lags)
X = X.reshape((X.shape[0], 1, X.shape[1]))
y = y.reshape((y.shape[0], 1))

# 16 units uit grid search 
model = Sequential([
    Input(shape=(X.shape[1], X.shape[2])),
    LSTM(50, activation='relu', dropout=0.2, return_sequences=True),
    Dense(1)
])

# Epochs uit initele grid search
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=50, batch_size=32, verbose=0)

# Now test the model on the test set, first create the features for the test set
X_test, y_test = create_features(X_test_list, number_of_lags)
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
y_test = y_test.reshape((y_test.shape[0], 1))

# Recursively predict the test set
current_input = X_test[0].reshape((1, 1, X_test.shape[2]))
y_test_pred = []

for i in range(len(y_test)):
    new_y = model.predict(current_input, verbose=0)
    y_test_pred.append(new_y[0][0])
    current_input = np.append(current_input[:, :, 1:], new_y.reshape((1, 1, 1)), axis=2)

y_test_pred = np.array(y_test_pred).reshape(-1, 1)

from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_test, y_test_pred)
mae = mean_absolute_error(y_test, y_test_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")


# Plot the predictions vs actual values
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test.flatten(), mode='lines', name='Actual Values'))
fig.add_trace(go.Scatter(y=y_test_pred.flatten(), mode='lines', name='Predicted Values'))
fig.update_layout(title='LSTM Predictions vs Actual Values', xaxis_title='Index', yaxis_title='Value')
fig.show()




Mean Squared Error (MSE): 2766.98
Mean Absolute Error (MAE): 38.49


In [43]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test.flatten(), mode='lines', name='Actual Values'))
fig.add_trace(go.Scatter(y=y_test_pred.flatten(), mode='lines', name='Predicted Values'))

fig.update_layout(
    xaxis_title='Time',
    yaxis_title='Value',
    legend=dict(
        x=0.98,
        y=0.98,
        xanchor='right',
        yanchor='top',
        bgcolor='rgba(255, 255, 255, 0.7)',  # Optional: translucent white background
        bordercolor='black',
        borderwidth=1
    )
)

fig.show()
